In [ ]:
# Temporal Difference Learning Model for Multiple Files in a Directory
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import os
from tqdm import tqdm

# Directory containing data
data_dir = r"C:\Users\boba4\Documents\GitHub\ucla_analysis\DAPL\data\FC\D0"
output_dir = os.path.join(data_dir, "analysis_results")
os.makedirs(output_dir, exist_ok=True)

# Function to create shocks array
def create_shocks_array(length, ranges):
    shocks = np.zeros(length)
    for start, end in ranges:
        shocks[start:end+1] = 1
    return shocks

# Master data containers
master_errors = []
master_da_signals = []
results_summary = []

# Loop through all CSV files in the directory
for file_name in tqdm(os.listdir(data_dir), desc="Processing files"):
    if file_name.endswith(".csv"):
        file_path = os.path.join(data_dir, file_name)
        dopamine_signals = pd.read_csv(file_path, header=None).to_numpy()
        num_trials = dopamine_signals.shape[0]
        time_steps = np.arange(dopamine_signals.shape[1])
        shock_ranges = [(1650, 1800)]  # Define shock ranges
        
        # Prepare output paths
        subject_dir = os.path.join(output_dir, os.path.splitext(file_name)[0])
        os.makedirs(subject_dir, exist_ok=True)
        
        all_params = []
        all_errors = []
        trial_data = []
        
        # Model fitting and analysis for each trial
        for trial in range(num_trials):
            average_DA_signal = dopamine_signals[trial, :]
            shocks = create_shocks_array(len(average_DA_signal), shock_ranges)

            def value_function(t, params):
                return params[0] * t + params[1]

            def td_error(t, params, shocks, gamma, alpha):
                V_t = value_function(t, params)
                V_t1 = value_function(t + 1, params)
                td_error = shocks[t] + gamma * V_t1 - V_t
                return V_t + alpha * td_error

            def likelihood(params):
                params_without_gamma_alpha = params[:-2]
                gamma = params[-2]
                alpha = params[-1]
                errors = np.array([
                    td_error(t, params_without_gamma_alpha, shocks, gamma, alpha) for t in time_steps[:-1]
                ])
                likelihoods = -0.5 * np.sum((average_DA_signal[1:] - errors) ** 2)
                return -likelihoods

            initial_params = np.array([0.1, 0.1, 0.3, 0.5])
            bounds = [(None, None), (None, None), (0, 1), (0, 1)]
            result = minimize(likelihood, initial_params, method='L-BFGS-B', bounds=bounds)
            optimal_params = result.x

            # Record parameters
            all_params.append(optimal_params)

            # Generate predicted signals
            predicted_signals = np.zeros_like(average_DA_signal)
            predicted_signals[0] = average_DA_signal[0]
            for t in range(len(average_DA_signal) - 1):
                predicted_signals[t + 1] = td_error(t, optimal_params[:-2], shocks, optimal_params[-2], optimal_params[-1]) + average_DA_signal[t]

            # Calculate error measures
            mse = np.mean((average_DA_signal - predicted_signals) ** 2)
            rmse = np.sqrt(mse)
            nrmse = rmse / (average_DA_signal.max() - average_DA_signal.min())
            r2 = 1 - (np.sum((average_DA_signal - predicted_signals) ** 2) / 
                      np.sum((average_DA_signal - average_DA_signal.mean()) ** 2))
            
            all_errors.append({
                "Trial": trial + 1,
                "MSE": mse,
                "NRMSE": nrmse,
                "R2": r2
            })

            # Save trial data
            trial_data.append({
                "Trial": trial + 1,
                "Observed": average_DA_signal,
                "Predicted": predicted_signals
            })
            
            # Append to master DA signals
            master_da_signals.append(pd.DataFrame({
                "Subject": file_name,
                "Trial": trial + 1,
                "Time Step": range(len(average_DA_signal)),
                "Observed": average_DA_signal,
                "Predicted": predicted_signals
            }))
        
        # Save parameters and errors
        params_df = pd.DataFrame(all_params, columns=["Time Coeff", "Baseline", "Gamma", "Alpha"])
        errors_df = pd.DataFrame(all_errors)
        params_df.to_csv(os.path.join(subject_dir, "parameters.csv"), index=False)
        errors_df.to_csv(os.path.join(subject_dir, "errors.csv"), index=False)
        
        # Append to master errors
        master_errors.append(errors_df.assign(Subject=file_name))
        
        # Append summary for review
        avg_errors = errors_df[["MSE", "NRMSE", "R2"]].mean()
        results_summary.append({
            "File": file_name,
            "Average MSE": avg_errors["MSE"],
            "Average NRMSE": avg_errors["NRMSE"],
            "Average R2": avg_errors["R2"],
            "Trials": num_trials
        })

# Combine master files
master_errors_df = pd.concat(master_errors, ignore_index=True)
master_da_signals_df = pd.concat(master_da_signals, ignore_index=True)

# Save master files
master_errors_file = os.path.join(output_dir, "master_errors.csv")
master_da_signals_file = os.path.join(output_dir, "master_da_signals.csv")
master_errors_df.to_csv(master_errors_file, index=False)
master_da_signals_df.to_csv(master_da_signals_file, index=False)

# Save summary of results
summary_df = pd.DataFrame(results_summary)
summary_file = os.path.join(output_dir, "summary.csv")
summary_df.to_csv(summary_file, index=False)

# Output completion message
print(f"Analysis complete. Results saved in: {output_dir}")


Processing files: 100%|██████████| 6/6 [00:41<00:00,  6.95s/it]


Analysis complete. Results saved in: C:\Users\boba4\Documents\GitHub\ucla_analysis\DAPL\data\yoked\analysis_results


In [31]:
summary_df

,File,Average MSE,Average NRMSE,Average R2,Trials
0,DP116_allTrial_signal.csv,16.487398,0.337255,-1.624299,27
1,DP117_allTrial_signal.csv,6.514119,0.187555,0.089712,27
2,DP118_allTrial_signal.csv,5.994685,0.175739,-0.063074,27
3,DP119_allTrial_signal.csv,9.602420,0.166177,-0.084979,27
4,DP121_allTrial_signal.csv,4.073017,0.180003,-0.192798,27
